Train 데이터 셋 분할

In [ ]:
# import os
# import pandas as pd
# from sklearn.model_selection import train_test_split

# DATA_DIR = '/data'
# ORIGIN_TRAIN_PATH = os.path.join(DATA_DIR, 'train.csv')
# SEED = 42


# df = pd.read_csv(ORIGIN_TRAIN_PATH)
# print(f"원본 데이터 개수: {len(df)}개")

# train_val, local_test = train_test_split(
#     df, test_size=0.1, stratify=df['label'], random_state=SEED
# )

# train, valid = train_test_split(
#     train_val, test_size=0.11, stratify=train_val['label'], random_state=SEED
# )

# print(f"분할 완료: Train({len(train)}) / Valid({len(valid)}) / Local_Test({len(local_test)})")

# train.to_csv(os.path.join(DATA_DIR, 'train_fixed.csv'), index=False)
# valid.to_csv(os.path.join(DATA_DIR, 'valid_fixed.csv'), index=False)
# local_test.to_csv(os.path.join(DATA_DIR, 'local_test_fixed.csv'), index=False)

# print("데이터 분할 및 저장 완료")

특수 기호 제거 로직

In [10]:
import os
import re
import pandas as pd
from pathlib import Path

# ==========================================
#  1. 경로 및 설정 (pathlib 적용)
# ==========================================
# 현재 작업 경로(Project Root)를 기준으로 경로를 잡음. 정상적인 경우 make-model이 베이스폴더가 됨
PROJECT_ROOT = Path.cwd()

# 입력 데이터 경로 (원본)
INPUT_DIR = PROJECT_ROOT / 'data' / 'raw'
INPUT_FILE_NAME = 'train.csv'

# 출력 데이터 경로 (저장용) - interim 폴더로 분리
OUTPUT_DIR = PROJECT_ROOT / 'data' / 'interim'
OUTPUT_FILE_NAME = 'clean_train.csv'

# ==========================================
#  2. Cleaning 함수 정의
# ==========================================
def clean_text(text):
    if not isinstance(text, str):
        text = str(text)

    # 1. HTML 태그 제거
    text = re.sub(r'<[^>]+>', '', text)

    # 2. 한자 제거
    text = re.sub(r'[\u4E00-\u9FFF]', '', text)

    # 3. 빈 괄호 및 내용 없는 괄호 패턴 제거
    text = re.sub(r'\(\s*[^\w가-힣]*\s*\)', '', text)

    # 4. ( ? ~ ? ) 형태의 특정 노이즈 패턴 제거
    text = re.sub(r'\([^\(\)]{0,20}[\?\~]{1,3}[^\(\)]{0,20}\)', '', text)

    # 5. 반복되는 점(...) 정제 -> 점(.) 하나로 변경
    text = re.sub(r'[.,]{3,}', '.', text)

    # 6. 반복되는 쉼표(,,,) 정제 -> 쉼표(,) 하나로 변경
    text = re.sub(r',\s*,+', ',', text)

    # 7. 중복 괄호((())) 정제 -> 괄호 하나로 변경
    text = re.sub(r'\({2,}', '(', text)
    text = re.sub(r'\){2,}', ')', text)

    # 8. 다중 공백 제거 (하나의 공백으로)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# ==========================================
#  3. 데이터 로드 함수
# ==========================================
def load_data(path):
    if not path.exists():
        print(f"파일이 존재하지 않음: {path}")
        return None
    
    encodings = ['utf-8', 'utf-8-sig', 'cp949']
    df = None
    
    for enc in encodings:
        try:
            df = pd.read_csv(path, encoding=enc)
            print(f"데이터 로드 성공 ({enc}): {path.name}")
            break
        except Exception:
            continue
            
    if df is None:
        print("데이터 로드 실패 (인코딩 문제일 수 있음)")
    return df

# ==========================================
#  4. 메인 실행 함수
# ==========================================
def main():
    input_path = INPUT_DIR / INPUT_FILE_NAME
    
    print("-" * 40)
    print(f"읽어올 경로: {input_path}")
    
    df = load_data(input_path)
    if df is None:
        return

    text_col = 'full_text' 
    if 'full_text' not in df.columns:
        for candidate in ['text', 'paragraph_text', 'content']:
            if candidate in df.columns:
                text_col = candidate
                break
    
    print(f"텍스트 컬럼 감지됨: '{text_col}'")
    print("데이터 정제 시작...")

    # 3. [리포트용] 변경 전 데이터 백업
    original_col = f"{text_col}_original"
    df[original_col] = df[text_col]

    # 4. 정제 수행
    df[text_col] = df[text_col].apply(clean_text)

    # 5. [리포트용] 실제로 내용이 바뀐 행만 추출
    changed_df = df[df[original_col] != df[text_col]]
    
    # --- 결과 리포트 출력 ---
    print("\n" + "="*30)
    print("      정제 결과 리포트      ")
    print("="*30)
    print(f"총 데이터 개수 : {len(df)}개")
    print(f"변경된 데이터  : {len(changed_df)}개 ({(len(changed_df)/len(df))*100:.2f}%)")
    
    if len(changed_df) > 0:
        print("\n[변경 예시 Top 3]")
        for i, row in changed_df.head(3).iterrows():
            print(f"전: {row[original_col]}")
            print(f"후: {row[text_col]}")
            print("-" * 20)
    else:
        print("\n(이미 깨끗해서 변경된 내용이 없습니다)")

    # 6. 빈 문자열 제거
    initial_len = len(df)
    df = df[df[text_col].str.strip() != ""]
    removed_count = initial_len - len(df)
    
    if removed_count > 0:
        print(f"\n정제 후 빈 문자열이 되어 제거된 행: {removed_count}개")

    # 7. 임시 컬럼 삭제
    df.drop(columns=[original_col], inplace=True)

    # 8. 저장 경로 설정 (폴더 없으면 자동 생성)
    output_path = OUTPUT_DIR / OUTPUT_FILE_NAME
    
    # 부모 폴더(processed)가 없으면 생성
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    
    # 저장
    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    
    print("\n" + "="*30)
    print(f"저장 완료!")
    print(f"저장 경로: {output_path}")
    print("="*30)

if __name__ == "__main__":
    main()

----------------------------------------
📂 읽어올 경로: c:\Users\Pyo\OneDrive\바탕 화면\git\make-model\data\raw\train.csv
✅ 데이터 로드 성공 (utf-8): train.csv
텍스트 컬럼 감지됨: 'full_text'
데이터 정제 시작...

      정제 결과 리포트      
총 데이터 개수 : 97172개
변경된 데이터  : 97041개 (99.87%)

[변경 예시 Top 3]
전: 카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5km2, 높이는 452m이다. 하와이 제도에서 가장 작은 화산섬이자 무인도이며 길이는 18km, 너비는 10km이다. 
 마우이섬에서 남서쪽으로 약 11km 정도 떨어진 곳에 위치하며 라나이섬의 남동쪽에 위치한다. 고도가 낮고 북동쪽에서 불어오는 무역풍을 통해 산악 지대에서 내리는 비를 형성하지 못하기 때문에 건조한 기후를 띤다. 마우이섬 화산의 비그늘에 속해 있기 때문에 섬 전체 면적의 1/4 이상이 부식되어 있다. 
 1000년경부터 사람이 거주했으며 해안 지대에는 소규모 임시 어촌이 형성되었다. 섬 안에는 주민들이 돌로 만든 제단에서 종교 의식을 거행한 흔적들, 주민들이 암석이나 평평한 돌에 그림을 그린 흔적들이 남아 있다. 1778년부터 1800년대까지 이 지역을 지나 항해하던 사람들의 보고에 따르면 카호올라웨섬은 무인도였고 나무도 물도 없는 불모지였다고 한다. 
 1830년대에는 하와이 왕국의 카메하메하 3세 국왕에 의해 남자 죄수들의 유형지로 사용되었지만 1853년에 폐지되었다. 1858년에는 하와이 정부가 목장 사업가들에게 카호올라웨섬을 양도했지만 가뭄과 과도한 방목으로 인해 땅이 말라갔다. 또한 강한 무역풍으로 인해 표토의 대부분이 날아가면서 붉은 경반층만 남게 되었다. 
 1910년부터 1918년까지 하와이 준주가 섬의 원래 모습을 복원하기 위해 이 섬을 천연보호구역으로 지정했지만 큰 성과를 거

4fold용 데이터셋 분할
1. train -> 0.9 : 0.1(train / test)
2. 4fold 0.75 : 0.25(train / valid)

In [11]:
import os
import pandas as pd
import csv
import numpy as np
from pathlib import Path
from sklearn.model_selection import StratifiedKFold, train_test_split

# ==========================================
#  1. 경로 및 설정 (pathlib 적용)
# ==========================================
PROJECT_ROOT = Path.cwd()

INPUT_DIR = PROJECT_ROOT / 'data' / 'interim'
INPUT_FILE_NAME = 'clean_train.csv'

# 출력 경로: Fold 데이터가 저장될 위치 ('data/fold')
OUTPUT_ROOT_DIR = PROJECT_ROOT / 'data' / 'fold'

# 저장될 파일 이름 설정
OUTPUT_TEST_FILENAME = 'local_origin_test.csv' 
OUTPUT_TRAIN_FILENAME = 'origin_train.csv'       
OUTPUT_VALID_FILENAME = 'origin_valid.csv'       

SEED = 42
detected_delimiter = ','
detected_quotechar = '"'

# ==========================================
#  2. 유틸리티 함수
# ==========================================

def find_column_name(columns, candidates):
    for col in columns:
        if col.lower().strip() in candidates:
            return col
    return None

def load_and_fix_data(path, is_test=False):
    # pathlib 객체 호환 처리
    if not path.exists():
        print(f"파일이 없음: {path}")
        return None

    df = None
    encodings_to_try = ['utf-8-sig', 'utf-8', 'cp949']

    for encoding in encodings_to_try:
        try:
            df = pd.read_csv(
                path,
                encoding=encoding,
                engine='python',
                on_bad_lines='skip',
                encoding_errors='ignore',
                delimiter=detected_delimiter,
                quotechar=detected_quotechar,
                quoting=csv.QUOTE_MINIMAL
            )
            break
        except Exception as e:
            df = None

    if df is None:
        print(f"데이터 로드 실패: {path}")
        return None

    # 컬럼명 정리 (text)
    text_candidates = ['paragraph_text', 'text', 'sentence', 'content', 'full_text']
    text_col = find_column_name(df.columns, text_candidates)
    if text_col:
        df.rename(columns={text_col: 'text'}, inplace=True)
    else:
        obj_cols = df.select_dtypes(include=['object']).columns
        if len(obj_cols) > 0:
            df.rename(columns={obj_cols[0]: 'text'}, inplace=True)
        else:
            return None

    # 컬럼명 정리 (id) - Test셋일 경우
    if is_test:
        id_candidates = ['id', 'idx', 'index', 'no', 'ID']
        id_col = find_column_name(df.columns, id_candidates)
        if id_col:
            df.rename(columns={id_col: 'id'}, inplace=True)
        else:
            df['id'] = df.index

    # 컬럼명 정리 (label) - Train셋일 경우
    if not is_test:
        target_candidates = ['generated', 'label', 'target', 'class']
        target_col = find_column_name(df.columns, target_candidates)
        if target_col:
            df.rename(columns={target_col: 'label'}, inplace=True)
            try:
                df['label'] = df['label'].astype(int)
            except:
                pass
        else:
            print("타겟(Label) 컬럼을 찾을 수 없습니다.")
            return None

    return df

def create_split_and_kfold(input_path, output_root, n_splits=4, test_size=0.1):
    print(f"-" * 40)
    print(f">>> 데이터 분할 작업 시작")
    print(f"입력 파일: {input_path}")
    print(f"출력 폴더: {output_root}")
    print(f"설정: Fold={n_splits}, Test Size={test_size}")
    print(f"-" * 40)
    
    # 1. 원본 데이터 로드
    df = load_and_fix_data(input_path, is_test=False)
    
    if df is None:
        print("데이터 로드 실패로 작업을 중단합니다.")
        return

    print(f"데이터 로드 완료: {len(df)}개 행")

    # 출력 루트 폴더 생성 (pathlib mkdir 사용)
    output_root.mkdir(parents=True, exist_ok=True)

    # 2. [Test Set 분리]
    print(f"\n[Step 1] Test Set 분리 (비율: {test_size})")
    dev_df, test_df = train_test_split(
        df, 
        test_size=test_size, 
        stratify=df['label'], 
        random_state=SEED,
        shuffle=True
    )
    
    # Test Set 저장
    test_save_path = output_root / OUTPUT_TEST_FILENAME
    test_df.to_csv(test_save_path, index=False)
    
    print(f" -> Test Set 저장 완료: {test_save_path.name}")
    print(f"    (Size: {len(test_df)}, Label 0: {(test_df['label']==0).sum()}, Label 1: {(test_df['label']==1).sum()})")

    # 3. [K-Fold 분할 및 폴더별 저장]
    print(f"\n[Step 2] {n_splits}-Fold Cross Validation 데이터 생성")
    
    dev_df = dev_df.reset_index(drop=True)
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)

    for fold, (train_idx, val_idx) in enumerate(skf.split(dev_df, dev_df['label'])):
        fold_train_df = dev_df.iloc[train_idx]
        fold_val_df = dev_df.iloc[val_idx]
        
        # 폴더 경로 생성 (fold0, fold1 ...)
        fold_dir_name = f'fold{fold}'
        current_fold_dir = output_root / fold_dir_name
        
        # 폴더 생성
        current_fold_dir.mkdir(parents=True, exist_ok=True)
        
        # 파일 저장
        train_save_path = current_fold_dir / OUTPUT_TRAIN_FILENAME
        val_save_path = current_fold_dir / OUTPUT_VALID_FILENAME
        
        fold_train_df.to_csv(train_save_path, index=False)
        fold_val_df.to_csv(val_save_path, index=False)
        
        print(f" -> [{fold_dir_name}] 저장 완료")
        # 상세 정보는 너무 길어질 수 있으니 필요하면 주석 해제
        # print(f"      Train: {len(fold_train_df)}개, Valid: {len(fold_val_df)}개")

    print("\n>>> 모든 작업이 완료됨.")
    print(f"최종 결과물 위치: {output_root}")

if __name__ == "__main__":
    # 실행 경로 조합
    input_full_path = INPUT_DIR / INPUT_FILE_NAME
    
    create_split_and_kfold(
        input_path=input_full_path, 
        output_root=OUTPUT_ROOT_DIR, 
        n_splits=4, 
        test_size=0.1
    )

----------------------------------------
>>> 데이터 분할 작업 시작
입력 파일: c:\Users\Pyo\OneDrive\바탕 화면\git\make-model\data\interim\clean_train.csv
출력 폴더: c:\Users\Pyo\OneDrive\바탕 화면\git\make-model\data\fold
설정: Fold=4, Test Size=0.1
----------------------------------------
데이터 로드 완료: 97172개 행

[Step 1] Test Set 분리 (비율: 0.1)
 -> Test Set 저장 완료: local_origin_test.csv
    (Size: 9718, Label 0: 8918, Label 1: 800)

[Step 2] 4-Fold Cross Validation 데이터 생성
 -> [fold0] 저장 완료
 -> [fold1] 저장 완료
 -> [fold2] 저장 완료
 -> [fold3] 저장 완료

>>> 모든 작업이 완료됨.
최종 결과물 위치: c:\Users\Pyo\OneDrive\바탕 화면\git\make-model\data\fold


데이터 축소 작업
Base: clean_train.csv
method: 